In [22]:
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:2


In [23]:
path = '/home/com_2/workspace/pig_dataset/test/*'
file_list = glob.glob(path)
files = sorted([file for file in file_list if file.endswith('.csv')])
print(len(files))
files

50


['/home/com_2/workspace/pig_dataset/test/20230504_14.csv',
 '/home/com_2/workspace/pig_dataset/test/20230504_18.csv',
 '/home/com_2/workspace/pig_dataset/test/20230504_7.csv',
 '/home/com_2/workspace/pig_dataset/test/20230511_1.csv',
 '/home/com_2/workspace/pig_dataset/test/20230511_3.csv',
 '/home/com_2/workspace/pig_dataset/test/20230511_6.csv',
 '/home/com_2/workspace/pig_dataset/test/20230518_2.csv',
 '/home/com_2/workspace/pig_dataset/test/20230518_7.csv',
 '/home/com_2/workspace/pig_dataset/test/20230622_13.csv',
 '/home/com_2/workspace/pig_dataset/test/20230622_16.csv',
 '/home/com_2/workspace/pig_dataset/test/20230622_17.csv',
 '/home/com_2/workspace/pig_dataset/test/20230622_20.csv',
 '/home/com_2/workspace/pig_dataset/test/20230706_4.csv',
 '/home/com_2/workspace/pig_dataset/test/20230720_14.csv',
 '/home/com_2/workspace/pig_dataset/test/20230720_15.csv',
 '/home/com_2/workspace/pig_dataset/test/20230720_16.csv',
 '/home/com_2/workspace/pig_dataset/test/20230727_1.csv',
 '/ho

In [24]:
def sliding_windows(data, seq_length):
    x = []
    y = []
    n = data.index[0]

    for i in range(len(data)-seq_length-1):
        _x = data[i:(i+seq_length)]

        # 학습 데이터셋 다음을 label로 (len(_x) + 1)
        _y = data.loc[n+seq_length]['label']
        n += 1

        # # 학습 데이터셋 마지막을 label로 (len(_x))
        # _y = data.loc[n+seq_length -1]['label']
        # n += 1

        x.append(_x.drop(columns=['label']))
        y.append([_y])
    

    return np.array(x),np.array(y)


In [25]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset): 
  def __init__(self, data_x, data_y):
    self.data_x = data_x
    self.data_y = data_y

  def __len__(self):
    return self.data_y.shape[0]

  def __getitem__(self, idx):
    x = torch.from_numpy(self.data_x[idx])
    y = torch.from_numpy(self.data_y[idx])
    return x,y
  
  def get_labels(self):
    return list(train_y_.squeeze())

In [26]:
n_hidden = 64
n_joints = 1
n_categories = 1
n_layer = 3
batch_size = 512
num_epochs = 150
learning_rate = 0.0001

In [29]:
class CRNN(nn.Module):
    def __init__(self, n_joints, n_hidden, n_categories, n_layer):
        super(CRNN, self).__init__()
        
        # CNN layers
        self.conv1 = nn.Conv1d(1, 32, kernel_size=1440, stride=1)
        self.relu1 = nn.ReLU()
        
        self.conv2 = nn.Conv1d(32, 64, kernel_size=1440, stride=1)
        self.relu2 = nn.ReLU()
        
        # LSTM layer
        self.lstm = nn.LSTM(64, n_hidden, num_layers=n_layer, batch_first=True, bidirectional=True)
        # self.lstm = nn.LSTM(64, n_hidden, num_layers=n_layer, batch_first=True)
        
        # FC layer
        self.fc = nn.Linear(n_hidden*2, n_categories)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = x.permute(0, 2, 1)  # 입력 데이터의 차원 변경
        x = self.conv1(x)
        x = self.relu1(x)
        
        x = self.conv2(x)
        x = self.relu2(x)
        
        x = x.permute(0, 2, 1)  # LSTM을 위해 차원 순서 변경
        x, _ = self.lstm(x)
        x = x[:, -1, :]  # 마지막 시간 스텝의 결과만 사용
        
        x = self.fc(x)
        x = self.sigmoid(x)
        
        return x

In [30]:
dir = '/home/com_2/workspace/yjshin/capstone/result_lstm/1013_230026_crnn_gt_1440_1440/100.pt'
file_name = dir.split('/')[-2] + '_' + dir.split('/')[-1]

seq_length = 2879

model = CRNN(n_joints, n_hidden, n_categories, n_layer)
model.load_state_dict(torch.load(dir))
model.to(device)


CRNN(
  (conv1): Conv1d(1, 32, kernel_size=(1440,), stride=(1,))
  (relu1): ReLU()
  (conv2): Conv1d(32, 64, kernel_size=(1440,), stride=(1,))
  (relu2): ReLU()
  (lstm): LSTM(64, 64, num_layers=3, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [33]:
from sklearn.metrics import f1_score

def test(model, test_loader):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        y_pred = np.array([])
        y_valid = np.array([])
        for inputs, labels in test_loader:
            inputs = inputs.to(device).float()
            labels = labels.to(device).float()
            labels = torch.squeeze(labels)
            # outputs = model(inputs).squeeze()
            # print(outputs)
            
            outputs = model(inputs)
            # predicted = torch.tensor([1 if x > 0.1 else 0 for x in outputs]).squeeze()
            predicted = torch.round(outputs).squeeze()
            # predicted = outputs.squeeze()

            # _, predicted = torch.max(outputs, 1)
            # y_pred.append(predicted)
            # y_valid.append(labels)

            y_pred = np.concatenate([y_pred, np.array(predicted.cpu())])
            y_valid = np.concatenate([y_valid, np.array(labels.cpu())])
            
            # y_pred = np.concatenate([y_pred, np.array(predicted)])
            # y_valid = np.concatenate([y_valid, np.array(labels.cpu())])
            total += labels.size(0)
            
    return y_valid, y_pred

In [34]:
def graph(pig_data, y_valid, y_pred, file, folder_name, seq_length):

        # gt = pig_data.index[pig_data['label'] == 1].tolist()[-1]
        fig = plt.figure(figsize=(12,5))
        ax1 = fig.add_subplot()

        window_size = 1440
        moving_average = []
        file_name = file.split('/')[-1][:-4]
        for i in range(len(pig_data['ear_state'])-window_size):
                moving_average.append(sum(pig_data['ear_state'][i:i+window_size])/window_size)

        window_size_2 = 720*2
        db_moving_average = []
        for i in range(len(moving_average)-window_size_2):
                db_moving_average.append((sum(moving_average[i:i+window_size_2]))/window_size_2)
        c1 = ax1.plot(pig_data['ear_state'].index[seq_length+1:], y_valid, label='gt')
        c2 = ax1.plot(pig_data['ear_state'].index[seq_length+1:], y_pred, label='pred')

        ax2 = ax1.twinx()
        # c3 = ax2.scatter(gt, db_moving_average[gt-(window_size+window_size_2)], c='r', zorder=3)
        c4 = ax2.plot(pig_data['ear_state'].index[window_size+window_size_2:], db_moving_average, color='gray', label='1440-720-MA')

        c = c1 + c2 + c4
        ax1.legend(c, ['gt', 'pred', '1440-1440-MA'], loc='upper right')
        plt.title(file_name)
        
        directory = f'/home/com_2/workspace/out_graph/{folder_name}'
        if not os.path.exists(directory):
                os.makedirs(directory)
        plt.savefig(f'{directory}/{file_name}.png')

In [35]:
def ma_graph(pig_data, y_valid, y_pred, file, folder_name, seq_length):
        fig = plt.figure(figsize=(12,5))
        ax1 = fig.add_subplot()

        file_name = file.split('/')[-1][:-4]
                
        window_size = 1440
        window_size_2 = 720*2
        
        
        ma_gt = np.convolve(y_valid, np.ones(window_size)/window_size, mode='valid')
        
        ma_pred = np.convolve(y_pred, np.ones(window_size)/window_size, mode='valid')

        ma_pig = pig_data["ear_state"].rolling(window=window_size).mean()
        ma_pig = ma_pig.dropna()
        
        db_ma_pig = ma_pig.rolling(window=window_size_2).mean()
        db_ma_pig = db_ma_pig.dropna()
        

        c1 = ax1.plot(pig_data['ear_state'].index[seq_length+window_size:], ma_gt, label='gt')
        c2 = ax1.plot(pig_data['ear_state'].index[seq_length+window_size:], ma_pred, label='pred')
        c3 = ax1.axhline(0.1, color='green', linestyle='--', linewidth=1)
        

        ax2 = ax1.twinx()
        c4 = ax2.plot(pig_data['ear_state'].index[window_size+window_size_2-2:], db_ma_pig, color='gray', label='1440-720-MA')

        c = c1 + c2 + c4
        ax1.legend(c, ['gt', 'pred', '1440-1440-MA'], loc='upper right')
        plt.title(file_name)
        
        directory = f'/home/com_2/workspace/out_graph/label_ma/{folder_name}_label_ma'
        if not os.path.exists(directory):
                os.makedirs(directory)
        plt.savefig(f'{directory}/{file_name}.png')

In [36]:
def ma_graph2(pig_data, y_valid, y_pred, file, folder_name, seq_length):
        fig = plt.figure(figsize=(12,5))
        ax1 = fig.add_subplot()

        file_name = file.split('/')[-1][:-4]
                
        window_size = 1440
        window_size_2 = 720*2
        
        
        ma_gt = np.convolve(y_valid, np.ones(window_size)/window_size, mode='valid')
        
        ma_pred = np.convolve(y_pred, np.ones(window_size)/window_size, mode='valid')

        ma_pig = pig_data["ear_state"].rolling(window=window_size).mean()
        ma_pig = ma_pig.dropna()
        
        db_ma_pig = ma_pig.rolling(window=window_size_2).mean()
        db_ma_pig = db_ma_pig.dropna()
        
        c1 = ax1.plot(pig_data['ear_state'].index[seq_length+1:],y_valid, label='gt')

        # c1 = ax1.plot(pig_data['ear_state'].index[seq_length+window_size:], ma_gt, label='gt')
        c2 = ax1.plot(pig_data['ear_state'].index[seq_length+window_size:], ma_pred, label='pred')
        c3 = ax1.axhline(0.1, color='green', linestyle='--', linewidth=1)
        

        ax2 = ax1.twinx()
        c4 = ax2.plot(pig_data['ear_state'].index[window_size+window_size_2-2:], db_ma_pig, color='gray', label='1440-720-MA')

        c = c1 + c2 + c4
        ax1.legend(c, ['gt', 'pred', '1440-1440-MA'], loc='upper right')
        plt.title(file_name)
        
        directory = f'/home/com_2/workspace/out_graph/label_ma/{folder_name}_label_ma2'
        if not os.path.exists(directory):
                os.makedirs(directory)
        plt.savefig(f'{directory}/{file_name}.png')

In [37]:
folder_name = f'{datetime.now().strftime("%m%d_%H%M%S")}_{file_name}'
print(folder_name)
for i, file in enumerate(files):
    print(file)
    pig_data = pd.read_csv(file)
    pig_data = pig_data.drop(columns=['Date_time'])
    x, y = sliding_windows(pig_data, seq_length)
    test_dataset = CustomDataset(x, y)
    test_loader = torch.utils.data.DataLoader(test_dataset,
                                            batch_size = batch_size,
                                            shuffle=False)
    y_valid, y_pred = test(model, test_loader)
    graph(pig_data, y_valid, y_pred, file, folder_name, seq_length)
    # ma_graph2(pig_data, y_valid, y_pred, file, folder_name, seq_length)
    # ma_graph(pig_data, y_valid, y_pred, file, folder_name, seq_length)

1016_151458_1013_230026_crnn_gt_1440_1440_100.pt
/home/com_2/workspace/pig_dataset/test/20230504_14.csv
/home/com_2/workspace/pig_dataset/test/20230504_18.csv
/home/com_2/workspace/pig_dataset/test/20230504_7.csv
